In [6]:
import duckdb
import requests
import time
con = duckdb.connect("data/db.duckdb")

In [7]:
validation_candidates = con.sql(
    """
    select
        co.email
    from crm.contacts as co
    left join crm.companies as c
        on co.company_id = c.id
    left join crm.customer_buying_intent as bi
        on bi.company_id = c.id
    where bi.buying_intent = 'HIGH'
    """
).to_df()

print(validation_candidates)

                                  email
0    alvarado.alvarado@becker-moore.net
1            love.love@becker-moore.net
2        holloway.holloway@crawford.com
3  gillespie.gillespie@pena-schmidt.com
4                  buck.buck@malone.com
5                lopez.lopez@malone.com
6              white.white@cummings.net
7       rodriguez.rodriguez@collins.com


In [8]:
#  email validation logic in python and back into CRM database.
api_key = 'b050890a542e41ef8e2e64eca20896d2'
email_list = validation_candidates['email'].tolist()

emails = []
for candidate_email in email_list:
  response = requests.get(f"https://emailvalidation.abstractapi.com/v1/?api_key={api_key}&email={candidate_email}")

  is_email_valid=False
  if response.status_code == 200 and response.json()['is_smtp_valid']["value"] == True and response.json()['is_mx_found']["value"] == True:
    is_email_valid = True
  emails.append((candidate_email, is_email_valid))
  time.sleep(1) # Sleep to not exceed request per second

print(emails)

[('alvarado.alvarado@becker-moore.net', False), ('love.love@becker-moore.net', False), ('holloway.holloway@crawford.com', True), ('gillespie.gillespie@pena-schmidt.com', False), ('buck.buck@malone.com', False), ('lopez.lopez@malone.com', False), ('white.white@cummings.net', True), ('rodriguez.rodriguez@collins.com', False)]


In [16]:
con.execute("""
    CREATE TABLE IF NOT EXISTS crm.valid_emails (
        email VARCHAR,
        is_email_valid VARCHAR
    )
""")

# inserting data 
con.executemany("INSERT INTO crm.valid_emails VALUES (?, ?)", emails)

con.sql("""
SELECT * FROM crm.valid_emails
"""
)

[('alvarado.alvarado@becker-moore.net', False), ('love.love@becker-moore.net', False), ('holloway.holloway@crawford.com', True), ('gillespie.gillespie@pena-schmidt.com', False), ('buck.buck@malone.com', False), ('lopez.lopez@malone.com', False), ('white.white@cummings.net', True), ('rodriguez.rodriguez@collins.com', False)]


┌──────────────────────────────────────┬────────────────┐
│                email                 │ is_email_valid │
│               varchar                │    varchar     │
├──────────────────────────────────────┼────────────────┤
│ alvarado.alvarado@becker-moore.net   │ false          │
│ love.love@becker-moore.net           │ false          │
│ holloway.holloway@crawford.com       │ true           │
│ gillespie.gillespie@pena-schmidt.com │ false          │
│ buck.buck@malone.com                 │ false          │
│ lopez.lopez@malone.com               │ false          │
│ white.white@cummings.net             │ true           │
│ rodriguez.rodriguez@collins.com      │ false          │
└──────────────────────────────────────┴────────────────┘

In [17]:
con.close()